# Lab 16 - Bootstrap and confidence intervals

### 16.1 Loading the data

For this lab, we will use data from a sample of babies born between 1960 and 1967 in California hospitals.  The data file is posted on Blackboard with this lab, but was originally from the [Stat Labs](https://www.stat.berkeley.edu/~statlabs/data/babies.data) website, which is currently down. 

This lab closely follows [Chapter 13.3 in Computational and Inferential Thinking: The Foundations of Data Science](https://www.inferentialthinking.com/chapters/13/3/Confidence_Intervals.html) by Ani Adhikari and John DeNero.

Open the data file in JupyterHub (by clicking on it), or in TextEdit (on a Mac), or in NotePad (on Windows). What do you notice about it?  How are the columns separated? 

The columns are: 

- bwt: Birth weight in ounces (999 unknown)
- gestation: Length of pregnancy in days (999 unknown)
- parity: 0= first born, 9=unknown
- age: mother's age in years
- height: mother's height in inches (99 unknown)
- weight: Mother's prepregnancy weight in pounds (999 unknown)
- smoke: Smoking status of mother:  0=not now, 1=yes now, 9=unknown

First, let's import the necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In the data file, the columns are separated by whitespace (ex. spaces, tabs, etc.) instead of by commas, as usual.  We can still use `pd.read_csv()` but need to add the parameter `sep = "\s+"` to tell the function that the columns are separated one or more whitespaces. 

Read `babies.data` into the dataframe `babies` using this parameter `sep = "\s+"` and check it worked correctly.

How big is the sample?  We will use this later.

<details> <summary>Pattern:</summary>
<code>
df.shape[0]</code>
</details>

## 95% confidence interval for median birth weight to number of gestational days ratio

### 16.2 Median number of gestational days

We would like to understand the relation between birth weight, which is given in ounces, and the number of gestational days.  Smaller babies often need more medical care, and it would be helpful to be able to estimate a baby's weight before it is born.

We'll look at the ratio of birth weight to number of gestational days.  For the first row, this ratio is:
$\frac{\text{120 oz}}{\text{284 days}} \approx \text{0.4225 ounces per day}$

Let's make a new column of the birth weight to number of gestational days ratio.  Recall from Lab 2 that we can do column-wise math operations.

<details> <summary>Answer</summary>
<code>
babies["Ratio BW/GD"] = babies["bwt"]/babies["gestation"]
</code>
</details>

Plot a histogram of these ratios:

We see in the histogram that there are some very low and very high ratios, which are *outliers*.  Because of these extreme values we should use a robust measure of center, namely the median.  Compute the median of the ratios.

<details> <summary>Pattern:</summary>
<code>
df["column_name"].median()
</code>
</details>

### 16.3 Bootstrapping and 95% Confidence Interval

This is the median ratio of the sample, but what is the median ratio of the population?  We will estimate a range for it, called the *95% confidence interval*, by creating more samples from the sample we have.  This process of creating more data with the same distribution as our existing data is called *bootstrapping*.

Above we found that our sample size is 1236.  Sampling *with replacement* means that we can choose the same row more than once (the row is replaced once we have chosen it). 

Steps for computing the 95% confidence interval:
1. Take a random sample of size 1236 from our sample, by sampling *with replacement*, and compute the median of the new sample.
2. Repeat Step 1 until we have computed 5000 sample medians, which we save in a list.
3. Compute the middle 95% of these medians.

####  Step 1: Take a random sample of size 1236 from our sample, by sampling *with replacement*, and compute the median of the new sample.

To re-sample with replacement from our original sample, we still use the `.sample()` function, but add the parameter `replace = True`.

Take a sample of size 1236 with replacement from the `babies` dataframe, and store it in the dataframe `bootstrap_sample`.

<details> <summary>Answer</summary>
<code>
bootstrap_sample = babies.sample(1236, replace = True)</code>
</details>

Check that `bootstrap_sample` was created correctly by displaying it.

Take the median ratio of the sample:

How does this median compare to the median of the original data?  The two medians should be similar.

#### Step 2:  Take 5,000 samples and compute the median for each as in Step 1.  Store these medians in a list.

This step is similiar to what we have done in previous labs, just with a different computation inside the loop.

The pseudo-code is:
<code>
 new_list = []
 for i in range(num_of_loops):
     take bootstrap sample
     compute median of ratio in bootstrap sample
     add median to new_list
</code>

Plot the medians as a histogram, remembering to convert the list into a Pandas Series first:

<details> <summary>Pattern:</summary>
<code>
pd.Series(median_list).hist()</code>
</details>

What do you notice about the distribution?  What is the range of the distribution?  

#### Step 3:  Compute the middle 95% of these medians.

To get the interval containing the middle 95% of our sample medians, we compute the 2.5 percentile, which is the sample median such that 2.5% of the sample medians are below it, and the 97.5 percentile, which is the sample median such that 97.5% of the sample medians are below it.

Pandas has a function for computing *quantiles* which are the percentiles divided by 100.  ie. the 2.5 percentile is the 0.025 quantile and the 97.5 percentile is the 0.975 quantile.

Type `pd.Series(medians).quantile(0.025)` below to compute the 0.025 quantile.

Now compute the 0.975 quantile:

The 95% confidence interval is [0.425,0.431] from my calculations.  Yours may differ slightly because we use random sampling to compute this interval.

Technically this interval is an *approximation* of the 95% confidence interval that you may have seen in another statistics class.

*Interpretation of the 95% confidence interval*: If we repeated this procedure for computing the 95% confidence interval on many different original samples from the population, then 95% of the intervals would contain the true population median.

This method for computing the 95% confidence interval also works for means and proportions.  It may not work well if:
- the sample size is small (say less than 10 or 15)
- the population distribution of the statistic is not roughly bell shaped
- we are trying to estimate a statistic that is greatly influenced by outliers or rare elements, like the maximum data value

## 95% confidence interval of the proportion of mothers who smoke

### 16.4 Proportion of mothers who smoke

Now we will use bootstrapping to compute the 95% confidence interval of the proportion of mothers who smoke.

First estimate the proportion of mothers who smoke in our sample, by computing:
$$\text{proportion of mothers who smoke} = \frac{\text{# of birth where mother smoked}}{\text{total # of births}}$$

Calculate the number of births where mother smoked:

<details> <summary>Hint:</summary>
Use a filter to find the rows with 1 in the `smoke` column.  
</details>

We computed the total number of births at the start of the lab.

Compute the proportion of mothers who smoke:

### 16.5 Bootstrap and 95% Confidence Interval

Now use bootstrapping to compute a 95% confidence interval for the proportion of mothers who smoke.

#### Step 1 and 2: Re-sample from the sample and compute the proportion of mothers who smoke for each bootstrap sample.

The pseudo-code is:
<code>
 new_list = []
 for i in range(num_of_loops):
     take bootstrap sample
     compute proportion of mothers who smoke in bootstrap sample
     add proportion to new_list
</code>

<details> <summary>Answer:</summary>
<code>
proportions = []
for i in range(5000):
    bootstrap_sample = babies.sample(1236, replace=True)
    smoke_filter = bootstrap_sample["smoke"] == 1
    num_smoke = smoke_filter.sum()
    proportion = num_smoke/1236
    proportions.append(proportion)
</code>    
</details>

Plot a histogram of the proportions, remembering to make the list into a Pandas Series first.

#### Step 3: Compute the 95% confidence interval for the proportions

Find the 0.025 quantile and the 0.975 quantile to get the interval.

<details> <summary>Pattern:</summary>
pd.Series(list_of_sample_statistic).quantile(desired_quantile)
</details>

What is your 95% confidence interval?

*Interpretation of the 95% confidence interval*: If we repeated this (random) procedure for computing the 95% confidence interval multiple times, then approximately 95% of the confidence intervals computed would contain the true proportion of mothers who smoke in the population.

### Challenges: 
- Find the 95% confidence interval for the age of the mothers in the sample.
- Find the 95% confidence interval for the mean birth weight in the sample.